In [2]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver



In [3]:
load_dotenv()
llm = ChatOpenAI()


In [7]:
class jokestate(TypedDict):
    
    topic : str
    joke: str
    explanation: str
    

In [8]:
def generate_joke(state: jokestate) -> jokestate:

    prompt = f"generate joke on the {state['topic']}."
    joke = llm.invoke(prompt).content
    return {'joke': joke}

In [9]:
def generate_explanation(state: jokestate) -> jokestate:
    prompt = f"Explain the joke: {state['joke']}"
    explanation = llm.invoke(prompt).content
    return {'explanation': explanation}

In [15]:
graph = StateGraph(jokestate)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)


graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)


In [20]:
config1 = {'configurable': {"thread_id": "1"}}
workflow.invoke({'topic': 'technology'}, config=config1)

{'topic': 'technology',
 'joke': 'Why did the computer go to the doctor? Because it had a bad case of a virus!',
 'explanation': 'This joke plays on the double meaning of the word "virus." In the context of computers, a virus refers to malicious software that can infect and harm a computer system. By saying the computer had a "bad case of a virus," it is suggesting that the computer is sick in a humorous way, as if it were a living being that could visit a doctor for treatment.'}

In [21]:
workflow.get_state(config=config1)

StateSnapshot(values={'topic': 'technology', 'joke': 'Why did the computer go to the doctor? Because it had a bad case of a virus!', 'explanation': 'This joke plays on the double meaning of the word "virus." In the context of computers, a virus refers to malicious software that can infect and harm a computer system. By saying the computer had a "bad case of a virus," it is suggesting that the computer is sick in a humorous way, as if it were a living being that could visit a doctor for treatment.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f094cfc-e5ff-66b6-8003-f65bf2c14390'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-09-18T20:41:09.002188+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f094cfc-d977-627c-8002-9427c5fc5e25'}}, tasks=(), interrupts=())

In [22]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'technology', 'joke': 'Why did the computer go to the doctor? Because it had a bad case of a virus!', 'explanation': 'This joke plays on the double meaning of the word "virus." In the context of computers, a virus refers to malicious software that can infect and harm a computer system. By saying the computer had a "bad case of a virus," it is suggesting that the computer is sick in a humorous way, as if it were a living being that could visit a doctor for treatment.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f094cfc-e5ff-66b6-8003-f65bf2c14390'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-09-18T20:41:09.002188+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f094cfc-d977-627c-8002-9427c5fc5e25'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'technology', 'joke': 'Why did the computer go to the doctor? Because it

In [23]:
config2 = {'configurable': {"thread_id": "2"}}
workflow.invoke({'topic': 'kasta'}, config=config2)

{'topic': 'kasta',
 'joke': 'Why did the kasta go to therapy? Because it was feeling a little emotionally unstable!',
 'explanation': 'This joke is a play on words with the word "kasta" sounding similar to the word "caste" which refers to a social class or ranking in society. The joke suggests that the kasta went to therapy because it was feeling emotionally unstable due to its social status or position in society. This is a humorous way of pointing out the idea that even inanimate objects or concepts like social hierarchies can have emotions and may need therapy to cope with them.'}

In [24]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'kasta', 'joke': 'Why did the kasta go to therapy? Because it was feeling a little emotionally unstable!', 'explanation': 'This joke is a play on words with the word "kasta" sounding similar to the word "caste" which refers to a social class or ranking in society. The joke suggests that the kasta went to therapy because it was feeling emotionally unstable due to its social status or position in society. This is a humorous way of pointing out the idea that even inanimate objects or concepts like social hierarchies can have emotions and may need therapy to cope with them.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f094d11-f4b1-6910-8002-6e876a69f1b0'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-18T20:50:34.257602+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f094d11-e525-6bd6-8001-1d373ac73730'}}, tasks=(), interrupts=())

Time Travel

In [25]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f094cfc-ce92-65d2-8001-1d0615d4e071"}})

StateSnapshot(values={'topic': 'technology'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f094cfc-ce92-65d2-8001-1d0615d4e071'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-09-18T20:41:06.545807+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f094cfc-ce90-6dc2-8000-47b236e479ae'}}, tasks=(PregelTask(id='a3a9d49c-433b-300e-2141-99dfd0736ef8', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': 'Why did the computer go to the doctor? Because it had a bad case of a virus!'}),), interrupts=())

In [26]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f094cfc-ce92-65d2-8001-1d0615d4e071"}})

{'topic': 'technology',
 'joke': 'Why was the computer cold?\nBecause it left its Windows open!',
 'explanation': 'This joke is based on a play on words. In reality, a computer cannot feel temperature, so it cannot actually be cold. However, the joke plays on the idea of a computer having "Windows" (referring to the operating system) open, which could let in cold air from the outside and make the computer feel cold. It is a humorous way to personify the computer and make a pun about the popular operating system.'}

In [27]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'technology', 'joke': 'Why was the computer cold?\nBecause it left its Windows open!', 'explanation': 'This joke is based on a play on words. In reality, a computer cannot feel temperature, so it cannot actually be cold. However, the joke plays on the idea of a computer having "Windows" (referring to the operating system) open, which could let in cold air from the outside and make the computer feel cold. It is a humorous way to personify the computer and make a pun about the popular operating system.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f094d39-6c59-6a54-8003-42a88b32834a'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-09-18T21:08:13.702784+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f094d39-5e38-6592-8002-53b10fbe61f5'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'technology', 'joke': 'Why was the co

In [28]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f094cfc-ce92-65d2-8001-1d0615d4e071", "checkpoint_ns": ""}}, {'topic':'samosa'})


{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f094d57-cf50-691e-8002-3e71ddee6210'}}

In [29]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f094d57-cf50-691e-8002-3e71ddee6210'}}, metadata={'source': 'update', 'step': 2, 'parents': {}}, created_at='2025-09-18T21:21:49.386359+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f094cfc-ce92-65d2-8001-1d0615d4e071'}}, tasks=(PregelTask(id='a286b8fa-f940-d1df-bd66-be341b835e62', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'technology', 'joke': 'Why was the computer cold?\nBecause it left its Windows open!', 'explanation': 'This joke is based on a play on words. In reality, a computer cannot feel temperature, so it cannot actually be cold. However, the joke plays on the idea of a computer having "Windows" (referring to the operating system) open, which could let in

In [30]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f094d57-cf50-691e-8002-3e71ddee6210"}})

{'topic': 'samosa',
 'joke': 'Why did the samosa go to the doctor?\nBecause it was feeling a little deep fried!',
 'explanation': 'This joke plays on the idea of food being "sick" or in need of medical attention. The samosa, a fried pastry with savory fillings, is jokingly portrayed as feeling "deep fried" in a literal and humorous sense, prompting it to seek help from a doctor. Overall, the joke is a play on words and a lighthearted way to personify the samosa.'}

In [31]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the samosa go to the doctor?\nBecause it was feeling a little deep fried!', 'explanation': 'This joke plays on the idea of food being "sick" or in need of medical attention. The samosa, a fried pastry with savory fillings, is jokingly portrayed as feeling "deep fried" in a literal and humorous sense, prompting it to seek help from a doctor. Overall, the joke is a play on words and a lighthearted way to personify the samosa.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f094d59-9f9c-6f54-8004-c1b4804348df'}}, metadata={'source': 'loop', 'step': 4, 'parents': {}}, created_at='2025-09-18T21:22:38.071455+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f094d59-96b8-60fa-8003-f8877f890b84'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the samosa go to the doctor?\nBecause it was feeling a little